## Install Libraries


In [21]:
%pip install "ray[tune]"
# %pip install torch torchvision torchaudio # Uncomment this to install PyTorch 2.0+ on ilab (required version to run this program)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Libraries


In [22]:
from filelock import FileLock
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from torch.utils.data import random_split
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from collections import OrderedDict


## Data loaders


In [23]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform
        )

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform
        )

    return trainset, testset


## Configurable neural network


In [24]:
def flatten(x, start_dim=1, end_dim=-1):
    return x.flatten(start_dim=start_dim, end_dim=end_dim)


In [25]:
# Will need to modify the parameters and the network depending on what you are experimenting with

C, H, W = 3, 32, 32
num_classes = 10

channel_1 = 32
channel_2 = 64
channel_3 = 128
pool_kernel_size = 2

kernel_size_1 = 3
kernel_size_2 = 5
kernel_size_3 = 7

pad_size_1 = 2
pad_size_2 = 3
pad_size_3 = 3

fc_count_1 = 1024

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, channel_1, kernel_size_1, padding=pad_size_1)
        self.norm1 = nn.BatchNorm2d(channel_1)
        self.pool1 = nn.MaxPool2d(pool_kernel_size)
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_size_2, padding=pad_size_2)
        self.norm2 = nn.BatchNorm2d(channel_2)
        self.pool2 = nn.MaxPool2d(pool_kernel_size)
        self.conv3 = nn.Conv2d(channel_2, channel_3, kernel_size_3, padding=pad_size_3)
        self.norm3 = nn.BatchNorm2d(channel_3)
        self.pool3 = nn.MaxPool2d(pool_kernel_size)        

        H_after_conv1 = int(H + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        W_after_conv1 = int(W + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        H_after_pool1 = int((H_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool1 = int((W_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        H_after_conv2 = int(H_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        W_after_conv2 = int(W_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        H_after_pool2 = int((H_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool2 = int((W_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)  
        H_after_conv3 = int(H_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        W_after_conv3 = int(W_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        H_after_pool3 = int((H_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool3 = int((W_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)

        self.fc1 = nn.Linear(H_after_pool3*W_after_pool3*channel_3, fc_count_1)
        self.fc2 = nn.Linear(fc_count_1, fc_count_1)
        self.fc3 = nn.Linear(fc_count_1, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = flatten(x)
        x = self.fc1(x)
        #x = F.relu(x)
        x = self.fc2(x)
        #x = F.relu(x)
        x = self.fc3(x)

        return x



## The train function


In [26]:
def train_cifar(config):
    net = Net()  # Will need to modify the parameters depending on what you are experimenting with

    learning_rate = config['learning_rate']
    momentum = config['momentum']
    weight_decay = config['weight']

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    

    criterion = nn.CrossEntropyLoss()

    if config['opt'] == 'sgd':
      optimizer = optim.SGD(net.parameters(), lr=learning_rate, 
                        weight_decay=weight_decay,
                        momentum=momentum, nesterov=False)
    
    elif config['opt'] == 'sgd_nes':
      optimizer = optim.SGD(net.parameters(), lr=learning_rate, 
                  weight_decay=weight_decay,
                  momentum=momentum, nesterov=True)
      
    elif config['opt'] == 'adagrad':
      optimizer = optim.Adagrad(net.parameters(), lr=learning_rate, 
                      weight_decay=weight_decay)
      
    elif config['opt'] == 'rms':
      optimizer = optim.RMSprop(net.parameters(), lr=learning_rate, 
                      weight_decay=weight_decay, momentum=momentum)
      
    elif config['opt'] == 'adam':
      optimizer = optim.Adam(net.parameters(), lr=learning_rate, 
                      weight_decay=weight_decay)

    #optimizer = optim.SGD(net.parameters(), lr=1e-2)

    # Do not modify any code below!
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=64, shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=64, shuffle=True, num_workers=8
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:
                print(
                    f"[{epoch + 1}, {i + 1:>5} loss: {running_loss / epoch_steps:.3f}]"
                )
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        os.makedirs("my_model", exist_ok=True)
        torch.save((net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report(
            {"loss": (val_loss / val_steps), "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")


## Test set accuracy


In [27]:
def test_best_model(best_result):
    best_trained_model = Net(
     )  # Will need to modify the parameters depending on what you are experimenting with

    # Do not modify any code below!
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(
        best_result.checkpoint.to_directory(), "checkpoint.pt"
    )

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Best trial test set accuracy: {correct / total}")


## Configuring the search space


In [28]:
# You will be experimenting with the hyperparameters here
# Use tune.grid_search to specify what values you want to experiment with a hyperparameter
config = {"opt": tune.grid_search(['sgd', 'sgd_nes', 'adagrad', 'rms', 'adam']), 
          "learning_rate": tune.grid_search([5e-2, 1e-2, 5e-3, 1e-3]),
          "momentum": tune.grid_search([0.001, 0.5]),
          "weight": tune.grid_search([1e-4, 1e-3, 1e-2, 1e-1]),
          }



In [29]:
# Do not modify any code below!
def main(config):
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"gpu": 1},
        ),
        param_space=config,
    )
    results = tuner.fit()

    best_result = results.get_best_result("accuracy", "max")

    print(f"Best trial config: {best_result.config}")
    print(f"Best trial final validation loss: {best_result.metrics['loss']}")
    print(f"Best trial final validation accuracy: {best_result.metrics['accuracy']}")

    test_best_model(best_result)

    return results.get_dataframe("accuracy", "max")


results_df = main(config)


(train_cifar pid=146349) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_19-42-04/train_cifar_19f07_00000_0_learning_rate=0.0500,momentum=0.0010,opt=sgd,weight=0.0001_2023-04-12_19-42-04/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:06<00:00, 28411777.30it/s]


(train_cifar pid=146349) Extracting /root/ray_results/train_cifar_2023-04-12_19-42-04/train_cifar_19f07_00000_0_learning_rate=0.0500,momentum=0.0010,opt=sgd,weight=0.0001_2023-04-12_19-42-04/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_19-42-04/train_cifar_19f07_00000_0_learning_rate=0.0500,momentum=0.0010,opt=sgd,weight=0.0001_2023-04-12_19-42-04/data
(train_cifar pid=146349) Files already downloaded and verified


Trial name,accuracy,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_cifar_19f07_00000,0.7486,2023-04-12_19-43-09,True,,34ae2ed95bb04015be831a8fb1c2936c,"0_learning_rate=0.0500,momentum=0.0010,opt=sgd,weight=0.0001",c167c01addae,10,0.909333,172.28.0.12,146349,True,62.075,5.01995,62.075,1681328589,0,,10,19f07_00000,0.00360274
train_cifar_19f07_00001,0.7319,2023-04-12_19-44-07,True,,34ae2ed95bb04015be831a8fb1c2936c,"1_learning_rate=0.0100,momentum=0.0010,opt=sgd,weight=0.0001",c167c01addae,10,0.872623,172.28.0.12,146349,True,57.9936,4.89372,57.9936,1681328647,0,,10,19f07_00001,0.00360274
train_cifar_19f07_00002,0.7262,2023-04-12_19-45-05,True,,34ae2ed95bb04015be831a8fb1c2936c,"2_learning_rate=0.0050,momentum=0.0010,opt=sgd,weight=0.0001",c167c01addae,10,0.824445,172.28.0.12,146349,True,58.1787,4.88972,58.1787,1681328705,0,,10,19f07_00002,0.00360274
train_cifar_19f07_00003,0.648,2023-04-12_19-46-03,True,,34ae2ed95bb04015be831a8fb1c2936c,"3_learning_rate=0.0010,momentum=0.0010,opt=sgd,weight=0.0001",c167c01addae,10,1.00392,172.28.0.12,146349,True,57.7595,4.81132,57.7595,1681328763,0,,10,19f07_00003,0.00360274
train_cifar_19f07_00004,0.7581,2023-04-12_19-47-01,True,,34ae2ed95bb04015be831a8fb1c2936c,"4_learning_rate=0.0500,momentum=0.5000,opt=sgd,weight=0.0001",c167c01addae,10,1.0135,172.28.0.12,146349,True,57.5179,4.82826,57.5179,1681328821,0,,10,19f07_00004,0.00360274
train_cifar_19f07_00005,0.7445,2023-04-12_19-47-59,True,,34ae2ed95bb04015be831a8fb1c2936c,"5_learning_rate=0.0100,momentum=0.5000,opt=sgd,weight=0.0001",c167c01addae,10,0.922588,172.28.0.12,146349,True,58.4177,4.90784,58.4177,1681328879,0,,10,19f07_00005,0.00360274
train_cifar_19f07_00006,0.7335,2023-04-12_19-48-57,True,,34ae2ed95bb04015be831a8fb1c2936c,"6_learning_rate=0.0050,momentum=0.5000,opt=sgd,weight=0.0001",c167c01addae,10,0.891789,172.28.0.12,146349,True,57.8271,4.87782,57.8271,1681328937,0,,10,19f07_00006,0.00360274
train_cifar_19f07_00007,0.6915,2023-04-12_19-49-55,True,,34ae2ed95bb04015be831a8fb1c2936c,"7_learning_rate=0.0010,momentum=0.5000,opt=sgd,weight=0.0001",c167c01addae,10,0.871162,172.28.0.12,146349,True,58.1002,4.79737,58.1002,1681328995,0,,10,19f07_00007,0.00360274
train_cifar_19f07_00008,0.7537,2023-04-12_19-50-54,True,,34ae2ed95bb04015be831a8fb1c2936c,"8_learning_rate=0.0500,momentum=0.0010,opt=sgd_nes,weight=0.0001",c167c01addae,10,0.891254,172.28.0.12,146349,True,58.4876,4.93914,58.4876,1681329054,0,,10,19f07_00008,0.00360274
train_cifar_19f07_00009,0.7405,2023-04-12_19-51-55,True,,34ae2ed95bb04015be831a8fb1c2936c,"9_learning_rate=0.0100,momentum=0.0010,opt=sgd_nes,weight=0.0001",c167c01addae,10,0.809856,172.28.0.12,146349,True,60.722,4.74896,60.722,1681329115,0,,10,19f07_00009,0.00360274


(train_cifar pid=146349) Finished Training
(train_cifar pid=146349) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_19-42-04/train_cifar_19f07_00001_1_learning_rate=0.0100,momentum=0.0010,opt=sgd,weight=0.0001_2023-04-12_19-43-09/data/cifar-10-python.tar.gz


...

(train_cifar pid=146349) Extracting /root/ray_results/train_cifar_2023-04-12_19-42-04/train_cifar_19f07_00158_158_learning_rate=0.0050,momentum=0.5000,opt=adam,weight=0.1000_2023-04-12_22-25-50/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_19-42-04/train_cifar_19f07_00158_158_learning_rate=0.0050,momentum=0.5000,opt=adam,weight=0.1000_2023-04-12_22-25-50/data
(train_cifar pid=146349) Files already downloaded and verified
(train_cifar pid=146349) Finished Training
(train_cifar pid=146349) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_19-42-04/train_cifar_19f07_00159_159_learning_rate=0.0010,momentum=0.5000,opt=adam,weight=0.1000_2023-04-12_22-26-53/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:05<00:00, 29290254.41it/s]


(train_cifar pid=146349) Extracting /root/ray_results/train_cifar_2023-04-12_19-42-04/train_cifar_19f07_00159_159_learning_rate=0.0010,momentum=0.5000,opt=adam,weight=0.1000_2023-04-12_22-26-53/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_19-42-04/train_cifar_19f07_00159_159_learning_rate=0.0010,momentum=0.5000,opt=adam,weight=0.1000_2023-04-12_22-26-53/data
(train_cifar pid=146349) Files already downloaded and verified
(train_cifar pid=146349) Finished Training


2023-04-12 22:27:56,349	INFO tune.py:798 -- Total run time: 9952.03 seconds (9951.95 seconds for the tuning loop).


Best trial config: {'opt': 'adam', 'learning_rate': 0.001, 'momentum': 0.001, 'weight': 0.001}
Best trial final validation loss: 0.7098283720244268
Best trial final validation accuracy: 0.7689
Files already downloaded and verified
Files already downloaded and verified
Best trial test set accuracy: 0.7395


In [30]:
final_df = results_df.drop(
    columns=[
        "should_checkpoint",
        "timesteps_total",
        "episodes_total",
        "training_iteration",
        "trial_id",
        "experiment_id",
        "date",
        "timestamp",
        "pid",
        "hostname",
        "node_ip",
        "time_since_restore",
        "timesteps_since_restore",
        "iterations_since_restore",
        "warmup_time",
        "logdir",
    ]
)
final_df.sort_values(by=['accuracy'], ascending=False).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/learning_rate,config/momentum,config/opt,config/weight
52,0.709013,0.7705,5.069118,False,44.510676,0.050,0.500,sgd_nes,0.0010
44,0.707999,0.7697,4.816374,False,44.206555,0.050,0.500,sgd,0.0010
75,0.676105,0.7693,5.235205,False,47.239297,0.001,0.001,adam,0.0010
79,0.742840,0.7651,5.419860,False,63.336574,0.001,0.500,adam,0.0010
85,0.681381,0.7646,5.078654,False,55.356371,0.010,0.500,sgd,0.0100
40,0.806759,0.7628,4.944818,False,60.141992,0.050,0.001,sgd,0.0010
67,0.720555,0.7621,5.093000,False,55.641869,0.001,0.001,rms,0.0010
48,0.719838,0.7611,4.919558,False,44.277488,0.050,0.001,sgd_nes,0.0010
12,0.829067,0.7608,4.998957,False,48.329915,0.050,0.500,sgd_nes,0.0001
4,0.703098,0.7601,4.720999,False,33.688661,0.050,0.500,sgd,0.0001


In [31]:
final_df.sort_values(by=['accuracy']).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/learning_rate,config/momentum,config/opt,config/weight
28,15489.728055,0.1013,14.860555,False,14.860555,0.050,0.500,rms,0.0001
29,133.256427,0.1036,4.899535,False,44.701674,0.010,0.500,rms,0.0001
32,2.582006,0.1055,5.300117,False,46.442156,0.050,0.001,adam,0.0001
36,2.401896,0.1058,5.093397,False,25.134995,0.050,0.500,adam,0.0001
68,5969.881981,0.1058,14.936442,False,14.936442,0.050,0.500,rms,0.0010
64,72.707830,0.1060,4.935476,False,24.813620,0.050,0.001,rms,0.0010
104,67.009581,0.1061,5.257035,False,35.994586,0.050,0.001,rms,0.0100
148,20.884484,0.1071,5.324552,False,41.266808,0.050,0.500,rms,0.1000
69,2.363841,0.1129,5.092666,False,40.040405,0.010,0.500,rms,0.0010
30,2.336396,0.1212,5.005960,False,39.970492,0.005,0.500,rms,0.0001
